# This notebook should serve as a tutorial for the various libraries I have built to run Tontine Analysis

In [1]:
import pandas as pd
import numpy as np
import longevity_lib as long
import matplotlib.pyplot as plt
import importlib as imp
import latexFormatLib as lf
from scipy import stats
import scipy
import time
import numbers
import options as opt

import stochastic_jump_generator as sjg
import self_directed_class as sdc
import individual_with_tontine_class as indton
import tontine_class as tonclass
import portfolio_weight_generator as pfg
import tontineCRRALib as efton

# I define basic parameters here

* age_0: age of retirement
* n_scen: number of retirement scenarios

In [2]:
age_0 = 65
n_scen = 1000

# In this tutorial, I generate asset returns via geometric brownian motion using the stochastic_jump_generator library

In [3]:
single_cov = pd.read_csv('Data/covariance_matrix_real.csv', index_col='Unnamed: 0')
asset_names = single_cov.columns
asset_names

Index(['US Equities', 'EAFE Equities', 'EM Equities', 'Leveraged Treasuries',
       'Corp Bonds', 'TIPS(backfilled)', 'Real Estate', 'TSMOMPos', 'T-bill'],
      dtype='object')

# These are the asset names, below are their expected monthly returns (although given annually and then divided by 12)

In [4]:
mean = np.array([3.13, 5.80, 7.19, .02, .69, -.36, 5.58, 1.08, -.90])/100./12.

In [5]:
imp.reload(sjg)
gbm_dist = sjg.gen_gbm_model(648, n_scen, mean, single_cov)

Gen Run i = 0
Gen Run i = 50
Gen Run i = 100
Gen Run i = 150
Gen Run i = 200
Gen Run i = 250
Gen Run i = 300
Gen Run i = 350
Gen Run i = 400
Gen Run i = 450
Gen Run i = 500
Gen Run i = 550
Gen Run i = 600
Gen Run i = 650
Gen Run i = 700
Gen Run i = 750
Gen Run i = 800
Gen Run i = 850
Gen Run i = 900
Gen Run i = 950


# Load Mortality Table
* This is a expected mortality table from the SSA

In [6]:
mortality_table = pd.read_csv('Data/Mortality_Table.csv', index_col='Exact Age')
m_mortality = mortality_table['Death Probability Male'].to_numpy()
f_mortality = mortality_table['Death Probability Female'].to_numpy()

In [7]:
mortality_table.head()

,Death Probability Male,Number of Lives Male,Life Expectancy Male,Death Probability Female,Number of Lives Female,Life Expectancy Female
Exact Age,,,,,,
0,0.006304,100000,75.97,0.005229,100000,80.96
1,0.000426,99370,75.45,0.000342,99477,80.39
2,0.000290,99327,74.48,0.000209,99443,79.42
3,0.000229,99298,73.50,0.000162,99422,78.43
4,0.000162,99276,72.52,0.000143,99406,77.45


# Generate portfolio weights using pfg library

In [8]:
imp.reload(pfg)
weights_30_fm = pfg.WeightCalculator('fixed mix', 65, 119, gender='M',
                               optional_dict={'equity proportion':.3, 'tontine proportion':0, 'n sim':1000}).target

weights_60_fm = pfg.WeightCalculator('fixed mix', 65, 119, gender='M',
                               optional_dict={'equity proportion':.6, 'tontine proportion':0, 'n sim':1000}).target

weights_100_fm = pfg.WeightCalculator('fixed mix', 65, 119, gender='M',
                               optional_dict={'equity proportion':1., 'tontine proportion':0, 'n sim':1000}).target

weights_gp = pfg.WeightCalculator('glide path', 65, 119, gender='M',
                               optional_dict={'equity proportion':1., 'tontine proportion of bond exposure':0,
                                              'n sim':1000}).target

# Load Realized Mortality

* Here I use longevity tables I've already generated using the mortality table.  You don't necessarily need the mortality_table defined above, except if you want to incorporate that information into your strategy
* For an individual who has access to a Tontine (one of the cases covered in this tutorial), you also need to specify a longevity table for the people in the tontine
* longevity tables have the following properties: columns are age, rows are the scenario, values are integers, encoding number of people alive at the start of the specific year.  For instance, the first column is always 10000 in this example.  If there are no people remaining in the tontine, the number is coded to -1.

In [9]:
death_table = pd.read_csv('CRRA_dt_runs/r_zero_runs/longevity_table_n_1.csv')
death_table.columns = [int(col) for col in death_table.columns]

In [10]:
tontine_table = pd.read_csv('CRRA_dt_runs/r_zero_runs/longevity_table_n_10000.csv')
tontine_table.columns = [int(col) for col in death_table.columns]

# Set Basic Parameters (v2)

* female_ind = 0: one can specify if people are male or female, is used in conjunction with the mortality_table to calculate probability of death
* w_0: amount of capital person starts with at the beginning of retirement
* start_of_annuity: optional argument, specifies beginning of annuity, assuming you assume the participant uses one
* annuity_info: prop: proportion of money paid into annuity paid out yearly (in real terms), second argument: amount of original capital placed into annuity (as a proportion)

In [11]:
female_ind= 0
#age_0 = 65
w_0 = 1000000
start_of_annuity = 80
prop = .183/(1.02**15)
#prop = .183
annuity_info = (prop, 1/((start_of_annuity - age_0)*prop + 1))

In [12]:
asset_names

Index(['US Equities', 'EAFE Equities', 'EM Equities', 'Leveraged Treasuries',
       'Corp Bonds', 'TIPS(backfilled)', 'Real Estate', 'TSMOMPos', 'T-bill'],
      dtype='object')

In [13]:
dist =  gbm_dist #two_regime_scenarios #gbm_dist 

# Since everything is calculated in real terms, I set my inflation matrix to be zero, but if you want to do everything in nominal terms, just adjust the return generating function, and generate a different inflation matrix inf_mat 

In [14]:
inf_mat = np.zeros((n_scen, 55))

# This will run a strategy with a person, but the individualWithTontine class can handle whatever strategy you want for an individual, with access to a yearly tontine.  I.e the person puts in money and at the end of the year, anyone who is alive receives 1/number of survivers of the money paid in

In order to run the code, you needs to specify a strategy dictionary, then call the .run_strat method.  The dictionary needs the following

d['Strategy Type']: 3 arguments, first is the name of the strategy, the second indicates the prescence of an annuity, the third allows you to overlay options strategies
You also need to fill out the ['Strategy_Specification'] value.  Please ask me questions if you are unsure about the library

In [15]:
imp.reload(indton)
names = ['Fixed Mix 30-70', 'Fixed Mix 60-40', 'Fixed Mix 100', 'Glide Path']
targets = [weights_30_fm, weights_60_fm, weights_100_fm, weights_gp]

test_individual = indton.individualWithTontine(death_table, tontine_table, dist, inf_mat, w_0, age_0, female_ind)


for i in range(len(names)):
    d = sdc.create_options_dict(targets[i], names[i])
    d['Strategy Type'] = ['twr_harmonic', 'annuity', None]
    d['Strategy Specification']['twr'] = .044
    d['Strategy Specification']['start_of_annuity'] = 80
    d['Strategy Specification']['amount_placed_in_annuity'] = annuity_info[1]
    d['Strategy Specification']['annuity_payout'] = prop
    test_individual.run_strat(d)

Beginning Run
Fixed Mix 30-70
Beginning Run
Fixed Mix 60-40
Beginning Run
Fixed Mix 100
Beginning Run
Glide Path


## Call the stack_tables method and you get a set of summary statistics

In [16]:
test_individual.stack_tables(names)

,Fixed Mix 30-70,Fixed Mix 60-40,Fixed Mix 100,Glide Path
Average Utility,-9.913080e+42,-1.029178e+45,-1.115597e+50,-8.825260e+42
Certainty Equivalent,2.930519e-02,2.496995e-02,1.674158e-02,2.942289e-02
Median Wealth at Death,1.411292e+05,2.626757e+05,3.985581e+05,1.334383e+05
Median Wealth at Start of Annuity,7.473968e+04,1.583127e+05,2.530742e+05,7.236737e+04
Median Wealth at Death Cond Die Before Annuity,3.377087e+05,4.066027e+05,4.738939e+05,3.385109e+05
Prob Ruin,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
Prob Ruin Cond Live to 80,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
Prob Ruin Cond Live to 90,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
Prob Fails TWR 0.02,0.000000e+00,3.000000e-03,1.900000e-02,0.000000e+00
Prob Fails TWR 0.025,3.000000e-03,7.000000e-03,6.800000e-02,3.000000e-03


# Lastly, one can easily write the table to a latex table (for easy copy / paste into a overleaf document)

In [17]:
#lf.write_table_fixed(test_individual.stack_tables(names), 'Path/To/Where/You/Save/filename.csv'
#                    'Title of Table', roundNumber=how many decimals you want to keep in your table)

# Let's also run a sample tontine analysis

you'll want to use tonclass.TONTINECLASS
 * first argument: longevity_table
 * second argument, dt_table: this specifies calculates the payout as a function of every dollar invested
 * third argument: dt_table: is the return information
 * inf_mat: inflation matrix
 * w_0: amount of money placed into the tontine by each person
 * gamma: longevity risk aversion parameter

In [18]:
dt_table = pd.read_csv('CRRA_dt_runs/r_zero_runs/dt_table_n_10000_gamma_30.csv')
dt_table.columns = [int(col) for col in dt_table.columns]

In [19]:
names

['Fixed Mix 30-70', 'Fixed Mix 60-40', 'Fixed Mix 100', 'Glide Path']

In [20]:
targets_2 = targets.copy()

In [21]:
for i in range(len(targets_2)):
    targets_2[i] = targets_2[i][:,0:9,0].copy()

## Unlike before, the targets numpy array that controls the asset allcation (of the money placed into the tontine) is a (55,9) array (not three dimensional in the case of an individual)

In [22]:
targets_2[0].shape

(55, 9)

In [23]:
imp.reload(tonclass)
gamma = 30
tontine_example = tonclass.TONTINECLASS(tontine_table, dt_table,
                                        gbm_dist, inf_mat, w_0, age_0, gamma=gamma)

for i in range(len(names)):
    d = sdc.create_options_dict(targets_2[i], names[i])
    d['Strategy Type'] = ['crra', 'no annuity', None]
    tontine_example.run_strat(d)

Beginning Run
Fixed Mix 30-70
Beginning Run
Fixed Mix 60-40
Beginning Run
Fixed Mix 100
Beginning Run
Glide Path


In [24]:
tontine_example.stack_tables(names)

In [25]:
tontine_example.performance_table

,Fixed Mix 30-70,Fixed Mix 60-40,Fixed Mix 100,Glide Path
Average Utility,-3.389500e+42,-7.130697e+42,-5.333235e+51,-3.363814e+42
Certainty Equivalent,3.040998e-02,2.964001e-02,1.465154e-02,3.041795e-02
Per Dollar Wealth Remaining At End of Tontine,4.651581e-04,8.965052e-04,2.101828e-03,3.285797e-04
Median Wealth at Start of Annuity,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
Median Wealth at Death Cond Die Before Annuity,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
Prob Ruin,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
Prob Ruin Cond Live to 80,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
Prob Ruin Cond Live to 90,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
Prob Fails TWR 0.02,0.000000e+00,3.800000e-05,1.212810e-02,1.340000e-05
Prob Fails TWR 0.025,7.278000e-04,3.060300e-03,3.901800e-02,1.805700e-03


# Now let's talk briefly about how to generate longevity tables and dt tables, below is sample code to calculate the dt_table (the payouts at each age dependent on the number of people, expressed as a proportion) and how to calculate the longevity_table

In [26]:
#longevity_table = efton.longevity_to_survivors(nscen,mortality_table, 1, age_0, n_in_tontine)
#dt_table = efton.calculate_dt_over_time(longevity_table, age_0, .0159, gamma, mortality_table, gender='M')